In [ ]:
import pandas as pd





In [ ]:
inventory_file = "/Users/mathieu.larouche.dube/sources/gomaterials/gomat-markup-optimization/data/GoMaterials_Historical_Inventory_20250225.csv"
df = pd.read_csv(inventory_file, encoding="utf-8")
df.head()

In [ ]:

# round the Updated_date to the nearest day
df["Updated_date"] = pd.to_datetime(df["Updated_date"], format="%Y-%m-%d %H:%M:%S").dt.floor("D")

# Group by supplier_id, supplier_name and Updated_date. Count the number of unique product_ids
df_grouped = df.groupby(["Supplier_id", "Supplier_name", "Updated_date"]).agg(
    distinct_product_ids=("Product_id", "nunique"),    
).reset_index()

# Sort the DataFrame by supplierid and Updated_date
df_grouped = df_grouped.sort_values(by=["Supplier_id", "Updated_date"])
df_grouped.head()


In [ ]:
# save the grouped DataFrame to a new CSV file
output_file = "/Users/mathieu.larouche.dube/sources/gomaterials/gomat-markup-optimization/data/exploration/GoMaterials_Historical_Inventory_Grouped.csv"
df_grouped.to_csv(output_file, index=False, encoding="utf-8")

In [ ]:
# count the number of distinct dates per supplier
supplier_dates = df_grouped.groupby(["Supplier_id", "Supplier_name"])["Updated_date"].nunique().reset_index()
supplier_dates.columns = ["Supplier_id", "Supplier_name", "Distinct_dates"]

# sort by Supplier_id
supplier_dates = supplier_dates.sort_values(by=["Supplier_id"])

# save the supplier_dates DataFrame to a new CSV file
output_file_dates = "/Users/mathieu.larouche.dube/sources/gomaterials/gomat-markup-optimization/data/exploration/GoMaterials_Historical_Inventory_Grouped_Dates.csv"
supplier_dates.to_csv(output_file_dates, index=False, encoding="utf-8")

In [ ]:
min_date = df_grouped["Updated_date"].min()
max_date = df_grouped["Updated_date"].max()

# create a date range from min_date to max_date with freqency of 1 week
date_range = pd.date_range(start=min_date, end=max_date, freq="W-SAT")

# for each date range, look back 90 days and count the number of distinct suppliers
date_range_df = pd.DataFrame(date_range, columns=["Updated_date"])
date_range_df["Distinct_suppliers"] = date_range_df["Updated_date"].apply(
    lambda x: df_grouped[
        (df_grouped["Updated_date"] >= x - pd.DateOffset(days=90))
        & (df_grouped["Updated_date"] <= x)
    ]["Supplier_id"].nunique()
)

# save the date_range_df DataFrame to a new CSV file
output_file_date_range = "/Users/mathieu.larouche.dube/sources/gomaterials/gomat-markup-optimization/data/exploration/GoMaterials_Historical_Inventory_Grouped_Date_Range.csv"

date_range_df.to_csv(output_file_date_range, index=False, encoding="utf-8")




In [ ]:
min_date = df["Updated_date"].min()
max_date = df["Updated_date"].max()

# create a date range from min_date to max_date with freqency of 1 week
date_range = pd.date_range(start=min_date, end=max_date, freq="W-SAT")

# for each date range, look back 90 days and count the number of distinct suppliers for each distinct region (column Supplier_region)
# then add 2 columns. Min region and max region. For example, if for a given date-90days, we have 3 suppliers in region A, 2 suppliers in region B and 0 suppliers in region C, we will add 2 columns. 
# Min_region = C and max_region = A

region_range_data = []
for x in date_range:
    subset = df.loc[(df['Updated_date'] >= x - pd.DateOffset(days=90)) & (df['Updated_date'] <= x)]
    region_counts = subset.groupby('Supplier_region')['Supplier_id'].nunique()
    if region_counts.empty:
        min_region, max_region = None, None
    else:
        min_region = region_counts.idxmin()
        max_region = region_counts.idxmax()
    region_range_data.append({
        'Updated_date': x,
        'Min_region': min_region,
        'Max_region': max_region,
        'Min_region_count': region_counts[min_region] if min_region else 0,
        'Max_region_count': region_counts[max_region] if max_region else 0
    })
region_range_df = pd.DataFrame(region_range_data)
region_range_df.head()
output_file_region_range = '/Users/mathieu.larouche.dube/sources/gomaterials/gomat-markup-optimization/data/exploration/GoMaterials_Historical_Inventory_Grouped_Region_Range.csv'
region_range_df.to_csv(output_file_region_range, index=False)